### Matplotlib Exercises###

In this exercise we will write a class which will take a pandas dataframe and output a set of plots describing the dataframe. For this we will use data from http://earthquake.usgs.gov/ which maintains csv files corresponding to data of earthquake occurrences sorted according to earthquakes magnitude, ranging from significant only down to all earthquakes, major or minor. The csv files are available here 

In [ ]:
feed="http://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/"

In [ ]:
import numpy as np
import pandas as pd
import urllib
import matplotlib as mpl
import matplotlib.pyplot as plt
%pylab inline

Check if you have the updated version of Matplotlib i.e 1.5.0. If not you can udate it using linux command sudo pip install --upgrade matplotlib for unix boxes and corresponding brew command for mac

In [ ]:
print mpl.__version__

### Acquiring Data ###

For this assignment we will be working with the following csv files

1. Significant earthquakes in the last 30 days : feed + "significant_month.csv"
2. Magnitude > 4.5 : feed + "4.5_month.csv"
3. Magnitude > 2.5 : feed + "2.5_month.csv"
4. Magnitude > 1.0 : feed + "1.0_month.csv"

Q1) Write a function which takes an integer input from 1 to 4 and outputs a pandas dataframe corresponding to the csv file above. Note that the function should necessarily download the data from the website via internet.

In [ ]:
feeds={1: feed + "significant_month.csv",
      2: feed+ "4.5_month.csv",
      3: feed+ "2.5_month.csv",
      4: feed + "1.0_month.csv"}

def download_data(i):
    return pd.read_csv(feeds[i])

In [ ]:
last_30 = download_data(1)
data_45 = download_data(2)
data_25 = download_data(3)
data_10 = download_data(4)

In [ ]:
data_10

###Q2) Analyzing the data ###

In order to understand the data we will plot different attributes of the data using tools provided by matplotlib.
In order to do that, we will first create a class object for the given dataframe.

2.1) The class will have a set of functions to generate different plots. The first function will output a histogram taking column type as input. The column type is restricted to latitude or longitude. Take default interval size as 10 degrees

2.2) Write another function which when called will output a scatter plot of magnitudes

In [ ]:
class dataPlot(object):
    """dataPlot takes a pandas dataframe as input and outputs different plots correspondng to the input dataframe"""
    
    def __init__(self, df):
        """Initialize variables"""
        self.df = df
        self.fig = plt.figure()
        
    def histogram(self,colType):
        ax1 = self.fig.add_subplot(2, 1, 1)         # 2x1 grid, first plot
        ax1.hist(self.df[colType], bins=15)
        
    def scatter(self):
        ax2 = self.fig.add_subplot(2, 1, 2)         # 2x1 grid, first plot
        ax2.scatter(self.df["longitude"], self.df["latitude"], s=self.df["mag"]*20)

In [ ]:
dp=dataPlot(data_10)
dp.histogram("latitude")
dp.scatter()

### Q3) Plotting data on the map ###

In the last class we had used basemap to plot the weather data. Now we will use basemap to plot the earthquake data, and further enhance it with a rain animation to indicate the severity of the incident. We will start initially with the dataset corresponding to earthquakes magnitude > 4.5

In [ ]:
data_45 = download_data(2)

First we will import necessary libraries

In [ ]:
from mpl_toolkits.basemap import Basemap
import matplotlib.animation as animation

Next we will generate a figure canvas with a basemap projection. We will use the 'mill' projection of Basemap for this assignment. 

In [ ]:
stations=data_45
lons=stations['longitude'].values
lats=stations['latitude'].values

In [ ]:
#General plot  initializations
fig = plt.figure(figsize=(14,10))
ax = plt.subplot(1,1,1)
earth = Basemap(projection='mill')
earth.drawcoastlines(color='0.50', linewidth=0.25)
earth.fillcontinents(color='0.95')


x, y = earth(lons,lats)
earth.plot(x,y,'g.')

We will then draw coastlines and continents

Now we will plot the magnitude of different earthquakes on the above plot according to the latitude and longitude of the origin and enhance it using Rain animation

#### Rain animation ####
A very simple rain effect can be obtained by having small growing rings randomly positioned over a figure. Of course, they won't grow forever since the wave is supposed to damp with time. To simulate that, we can use a more and more transparent color as the ring is growing, up to the point where it is no more visible. At this point, we remove the ring and create a new one.

An example from rain animation can be loaded by running the next cell (source: http://matplotlib.org/examples/animation/rain.html)

In [ ]:
# %load rain.py
"""Rain Simulation

Simulates rain drops on a surface by animating the scale and opacity
of 50 scatter points.

Author: Nicolas P. Rougier
"""
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation


# Create new Figure and an Axes which fills it.
fig = plt.figure(figsize=(7, 7))
ax = fig.add_axes([0, 0, 1, 1], frameon=False)
ax.set_xlim(0, 1), ax.set_xticks([])
ax.set_ylim(0, 1), ax.set_yticks([])

# Create rain data
n_drops = 50
rain_drops = np.zeros(n_drops, dtype=[('position', float, 2),
                                      ('size',     float, 1),
                                      ('growth',   float, 1),
                                      ('color',    float, 4)])

# Initialize the raindrops in random positions and with
# random growth rates.
rain_drops['position'] = np.random.uniform(0, 1, (n_drops, 2))
rain_drops['growth'] = np.random.uniform(50, 200, n_drops)

# Construct the scatter which we will update during animation
# as the raindrops develop.
scat = ax.scatter(rain_drops['position'][:, 0], rain_drops['position'][:, 1],
                  s=rain_drops['size'], lw=0.5, edgecolors=rain_drops['color'],
                  facecolors='none')


def update(frame_number):
    # Get an index which we can use to re-spawn the oldest raindrop.
    current_index = frame_number % n_drops

    # Make all colors more transparent as time progresses.
    rain_drops['color'][:, 3] -= 1.0/len(rain_drops)
    rain_drops['color'][:, 3] = np.clip(rain_drops['color'][:, 3], 0, 1)

    # Make all circles bigger.
    rain_drops['size'] += rain_drops['growth']

    # Pick a new position for oldest rain drop, resetting its size,
    # color and growth factor.
    rain_drops['position'][current_index] = np.random.uniform(0, 1, 2)
    rain_drops['size'][current_index] = 5
    rain_drops['color'][current_index] = (0, 0, 0, 1)
    rain_drops['growth'][current_index] = np.random.uniform(50, 200)

    # Update the scatter collection, with the new colors, sizes and positions.
    scat.set_edgecolors(rain_drops['color'])
    scat.set_sizes(rain_drops['size'])
    scat.set_offsets(rain_drops['position'])


# Construct the animation, using the update function as the animation
# director.
animation = FuncAnimation(fig, update, interval=10)
plt.show()



As you can see from the above code, the animation can be called from matplotlib's animation function using FuncAnimation or in our case animation.FuncAnimation. This will require an update function, a canvas fig, and an interval)

Q4) Now, based on the above rain animation, write an update function to plot the earthquake on earlier created fig. Set the interval as 10. i.e 
animation = animation.FuncAnimation(fig, update, interval=10)

To do this, create two objects rain_drops and scat as shown in the code above. The rain_drops and scat should be of the type as follows.

In [ ]:
from matplotlib.animation import FuncAnimation

In [ ]:
fig = plt.figure(figsize=(7, 7))
rain_drops = np.zeros(50, dtype=[('position', float, 2),
                         ('size',     float, 1),
                         ('growth',   float, 1),
                         ('color',    float, 4)])
scat = ax.scatter(rain_drops['position'][:,0], rain_drops['position'][:,1], rain_drops['size'], lw=0.5,
                  edgecolors = rain_drops['color'], facecolors='None', zorder=10)

Some of the key attributes for the plot should be set as follows
1. number of drops is 50
2. drop size is 5
3. growth is exponential with respect to magnitude i.e np.exp(magnitude) * 0.1
4. If magnitude less than 6 set color as 0,0,1,1 else set color as 1,0,0,1

The scat object functions 
1. set_edgecolors
2. set_sizes
3. set_offsets
4. set_faceColors

sets these values and returns the scat object. 

In [ ]:
def update(frame_number):
    # Get an index which we can use to re-spawn the oldest raindrop.
    current_index = frame_number % n_drops
    print current_index

    # Make all colors more transparent as time progresses.
    rain_drops['color'][:, 3] -= 1.0/len(rain_drops)
    rain_drops['color'][:, 3] = np.clip(rain_drops['color'][:, 3], 0, 1)

    rain_drops['size'] += rain_drops['growth']

    if(rain_drops['mag'][current_index]>6):
        rain_drops['color'][current_index] =  (1,0,0,1)

    # Update the scatter collection, with the new colors, sizes and positions.
    scat.set_edgecolors(rain_drops['color'])
    scat.set_sizes(rain_drops['size'])
    scat.set_offsets(rain_drops['position'])

Finally, plot the frames as an animation 

In [ ]:
animation = FuncAnimation(fig, update, interval=10)
plt.show()

#### Q4) Now, add this function to the above class dataPlot and call it for the dataframes data_25, data_1 ####

In [ ]:
class dataPlot(object):
    """dataPlot takes a pandas dataframe as input and outputs different plots correspondng to the input dataframe"""
    
    def __init__(self, df):
        """Initialize variables"""
        self.df = df
        self.fig = plt.figure()
        
    def histogram(self,colType):
        ax1 = self.fig.add_subplot(2, 1, 1)         # 2x1 grid, first plot
        ax1.hist(self.df[colType], bins=15)
        
    def scatter(self):
        ax2 = self.fig.add_subplot(2, 1, 2)         # 2x1 grid, first plot
        ax2.scatter(self.df["longitude"], self.df["latitude"], s=self.df["mag"]*20)
        
    def animate(self):
        lons = self.df['longitude'].values
        lats = self.df['latitude'].values

        # General plot  initializations
        self.fig = plt.figure(figsize=(14,10))
        ax = self.fig.add_subplot(1,1,1)
        earth = Basemap(projection='mill')
        earth.drawcoastlines(color='0.50', linewidth=0.25)
        earth.fillcontinents(color='0.95')
        x, y = earth(lons,lats)
        earth.plot(x,y,'g.')
        
        #rain

        # Create rain data
        n_drops = len(x)
        print n_drops
        rain_drops = np.zeros(n_drops, dtype=[('position', float, 2),
                                              ('size',     float, 1),
                                              ('growth',   float, 1),
                                              ('mag',      float, 1),
                                              ('color',    float, 4)])

        rain_drops['mag'] = self.df['mag']
        rain_drops['position'] = np.column_stack((x,y)) # n_drop x 2 matrix  #np.random.uniform(0, 1, (n_drops, 2)) *1000000
        rain_drops['color'] = (0,0,1,1)
        rain_drops['growth'] = np.exp(self.df['mag']) * 0.1 #n_drop randoms between 50 & 200 

        # Construct the scatter which we will update during animation
        # as the raindrops develop.
        scat = ax.scatter(rain_drops['position'][:, 0], rain_drops['position'][:, 1],
                          s=rain_drops['size'], lw=0.5, edgecolors=rain_drops['color'],
                          facecolors='none', zorder=100)


        def update(frame_number):
            # Get an index which we can use to re-spawn the oldest raindrop.
            current_index = frame_number % n_drops
            print current_index

            # Make all colors more transparent as time progresses.
            rain_drops['color'][:, 3] -= 1.0/len(rain_drops)
            rain_drops['color'][:, 3] = np.clip(rain_drops['color'][:, 3], 0, 1)

            rain_drops['size'] += rain_drops['growth']

            if(rain_drops['mag'][current_index]>6):
                rain_drops['color'][current_index] =  (1,0,0,1)

            # Update the scatter collection, with the new colors, sizes and positions.
            scat.set_edgecolors(rain_drops['color'])
            scat.set_sizes(rain_drops['size'])
            scat.set_offsets(rain_drops['position'])


        # Construct the animation, using the update function as the animation
        # director.
        animation = FuncAnimation(self.fig, update, interval=10)
        plt.show()


data_10 = download_data(2)
dataPlot(data_10).animate()